In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from model_robust import return_model
import os

In [2]:
args = {}
args['resume'] = False
args['lr'] = 0.01

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


def train(epoch, name):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx%100 == 0:
            print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'%(train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch,name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('cifar_resnet'):
            os.mkdir('cifar_resnet')
        torch.save(state, './cifar_resnet/ckpt_{}.pth'.format(name))
        best_acc = acc

namesss = ['convlinearl1k3c32relu']

for mm in namesss:
    # Model
    best_acc = 0
    print('==> Building model {}..'.format(mm))
    name = mm
    net = return_model(name)
    net = net.to(device)
    if device == 'cuda':
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=args['lr'], momentum=0.9)
    #optimizer = optim.Adam(net.parameters(), lr=args['lr'])

    lambda1 = lambda epoch: 0.3**((epoch)//4)
    scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])
    for epoch in range(start_epoch, start_epoch+40):
        train(epoch,name)
        test(epoch,name)
        scheduler.step()
    net = net.cpu()
    del net

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model convlinearl1k3c32relu..

Epoch: 0
0 391 Loss: 2.350 | Acc: 8.594% (11/128)
100 391 Loss: 1.956 | Acc: 30.059% (3886/12928)
200 391 Loss: 1.814 | Acc: 36.070% (9280/25728)
300 391 Loss: 1.729 | Acc: 39.195% (15101/38528)
99 100 Loss: 1.407 | Acc: 50.530% (5053/10000)
Saving..

Epoch: 1
0 391 Loss: 1.358 | Acc: 50.781% (65/128)


KeyboardInterrupt: 